In [ ]:
cnx = mysql.connector.connect(user='shawn', password='shawn672000',
                              host='CABOLLT0069',
                              database='enron')
g = cnx.cursor()

q =  "SELECT * FROM message"
msgs = pd.read_sql(con=cnx, sql=q, index_col="mid")

q = "SELECT * FROM recipientinfo"
recp = pd.read_sql(con=cnx, sql=q, index_col="rid")

msgs = msgs[msgs.sender.str.endswith("@enron.com")].reset_index(drop=True)

recp = recp[recp.rvalue.str.endswith("@enron.com")]

msgs = msgs.merge(recp, how="inner", left_index=True, right_on="mid")
msgs = msgs[["sender", "date", "mid", "subject", "rtype", "rvalue", "body"]]
msgs.columns = ["sender", "date", "mid", "subject", "type", "recipient", "body"]
msgs = msgs[msgs["recipient"] != "no.address@enron.com"]
msgs = msgs.reset_index(drop=True)
msgs = msgs[msgs.date.apply(lambda x:int(str(x)[:4])>=2000 and int(str(x)[:4])<=2002)]
msgs.reset_index(drop=True, inplace=True)
msgs["year"] = msgs.date.dt.year
msgs["month"] = msgs.date.dt.month
msgs["date_val"] = msgs.date.dt.date
msgs["month_name"] = msgs.date.dt.month_name()

msgs.recipient = msgs.recipient.apply(lambda x: x.split("@")[0].replace(".", " "))
msgs.sender = msgs.sender.apply(lambda x: x.split("@")[0].replace(".", " "))

msgs.to_pickle("enron_messages.pickle")

In [1]:
import numpy as np
import pandas as pd
import mysql.connector
import gc, os
import seaborn as sns
import networkx as nx
%matplotlib inline

# Read messages and create graph

In [ ]:
def build_undirected_graph()g = nx.Graph()
msgs = pd.read_pickle("enron_messages.pickle")
for idx in range(len(msgs)):
    snd = msgs.iloc[idx]["sender"]
    rec = msgs.iloc[idx]["recipient"]
    sub = msgs.iloc[idx]["subject"]
    yr = msgs.iloc[idx]["year"]
    m = msgs.iloc[idx]["month"]
    mn = msgs.iloc[idx]["month_name"]
    tp = msgs.iloc[idx]["type"]
    g.add_edge(snd, rec, subject=sub, year=str(yr), month=str(m), mtype=tp)

    
    
nx.write_gml(G=g, path="enron.gml")

In [18]:
g = nx.read_gml(path="enron.gml")

In [19]:
def trim_edges(g, degree):
    g2 = g.copy()
    d = g.degree()
    for n in g.nodes():
        if d[n]<=degree:
            g2.remove_node(n)
    return(g2)
            
    


In [27]:
core5 = trim_edges(g, degree=5)
core200 = trim_edges(g, degree=200)


686

In [ ]:
dg = 
dc = pd.Series(nx.degree_centrality(core5), name="degree")
clc = pd.Series(nx.closeness_centrality(core5), name="closeness")
bnc = pd.Series(nx.betweenness_centrality(core5), name="betweenness")
eigc = pd.Series(nx.eigenvector_centrality(core5), name="eigen")
all_metrics = pd.concat([dc, clc, bnc, eigc], axis=1)
all_metrics.to_pickle("all_metrics.pickle")

nx.set_node_attributes(G=core5, name="degree", values=all_metrics.degree.to_dict())
nx.set_node_attributes(G=core5, name="betweenness", values=all_metrics.betweenness.to_dict())
nx.set_node_attributes(G=core5, name="closeness", values=all_metrics.closeness.to_dict())
nx.set_node_attributes(G=core5, name="eigen", values=all_metrics.eigen.to_dict())

nx.write_gml(G=core5, path="core5.gml")

Kenneth Lay, Jeffrey Skilling, Richard Causey, Tim Belden, ken skilling

In [2]:
all_metrics = pd.read_pickle('all_metrics.pickle')
core5 = nx.read_gml(path="core5.gml")

In [61]:
ken_ego = nx.ego_graph(G=core5, n="ken lay", radius=1)
tim_ego = nx.ego_graph(G=core5, n="tim belden", radius=1, center=True)

In [63]:
nx.write_gml(G=tim_ego, path="tim_ego.gml")

In [30]:
cliques = list(nx.find_cliques(G=core200))

KeyboardInterrupt: 

True